In [50]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('./scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from certificate import Certificate
from blockchain import Blockchain
from smart_contract import SmartContractDefinition, SmartContractWritingOperation
from network import Node
from node import BlockchainNode
from wallet import Wallet
from proof_of_stake import ProofOfStake
from block import Block
from pebble import Account
from time import sleep
from helpers import timestamp

We'll start by generating the wallets for supposedly the "users".
In our case, we'll also be giving each user an account (assuming they've registrated to the betting website).

The account also contains the amount of pebbles a user has.

In [52]:
walletNode1=Wallet()
walletNode2=Wallet()
walletNode3=Wallet()

Bob=Wallet()
BobAccount = Account(Bob.publicKey)
print('Bob has ', BobAccount.pebbleAmount, ' pebbles')
John=Wallet()
JohnAccount = Account(John.publicKey)
print('John has ', JohnAccount.pebbleAmount, ' pebbles')
Alice=Wallet()
AliceAccount = Account(Alice.publicKey)
print('Alice has ', AliceAccount.pebbleAmount, ' pebbles')
Emma=Wallet()
EmmaAccount = Account(Alice.publicKey)
print('Emma has ', EmmaAccount.pebbleAmount, ' pebbles')

pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)

Bob has  20  pebbles
John has  20  pebbles
Alice has  20  pebbles
Emma has  20  pebbles


In [53]:
def read_text_file(fileName):
    with open(fileName, 'r', encoding='utf-8') as f:
        return f.read()
    
betFilePath="./scripts/bet.py"    

We'll start by assuming that Alice has generated a bet.
John and Emma see the bet and want to join in.

In [54]:
reset_blockchain()

bet1=SmartContractDefinition(Alice.publicKey,read_text_file(betFilePath))
Alice.sign(bet1)
node1.new_certificate(bet1)
sleep(1)

bet1_start = SmartContractWritingOperation(Alice.publicKey, bet1.hash(), 'startBet', [timestamp.now()+timestamp.now(), 'Lorum Lora', ['yes', 'no', 'maybe']])
Alice.sign(bet1_start)
node1.new_certificate(bet1_start)
sleep(1)

cert1 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert1)
node1.new_certificate(cert1)
sleep(1)

cert2 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 2])
John.sign(cert2)
node1.new_certificate(cert2)
sleep(1)

cert3 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 10, 1])
John.sign(cert3)
node1.new_certificate(cert3)
sleep(1)


cert4 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert4)
node1.new_certificate(cert4)
sleep(1)

cert5 = SmartContractWritingOperation(Emma.publicKey, bet1.hash(), 'addBetter', [EmmaAccount, 2, 3])
Emma.sign(cert5)
node1.new_certificate(cert5)
sleep(1)

cert6 = SmartContractWritingOperation(Emma.publicKey, bet1.hash(), 'addBetter', [EmmaAccount, 2, 3])
Emma.sign(cert6)
node1.new_certificate(cert6)
sleep(1)

cert7 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert7)
node1.new_certificate(cert7)
sleep(1)

node2.blockchain.display()


{'Genesis Key': '00000000', 'Genesis Index': 0}
{'Public Key': '30820122', 'Signature': '724c9c4c', 'timestamp': 1709073278953, 'type': <class 'block.Block'>}
{'Index': 1, 'Parent hash': 'f9e5d8b4'}
--Block certs--
{'Public Key': '30820122', 'Signature': 'b8d20c69', 'timestamp': 1709073274711, 'type': <class 'smart_contract.SmartContractDefinition'>}
{'Public Key': '30820122', 'Signature': 'a1aacd50', 'timestamp': 1709073277863, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': 'a7fd7049', 'timestamp': 1709073275758, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '00c4f179', 'timestamp': 1709073276811, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '0ddb0eb4', 'timestamp': 1709073278915, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
----


In [55]:
print(read_text_file(betFilePath))

class SmartContract():
    def __init__(self,ownerPublicKey):
        self.ownerPublicKey=ownerPublicKey
        self.description=None
        self.startTime=None
        self.closingPeriod=None
        #dictionnary of betters with their public key, the amount staked, and the outcome they want
        self.betters=[]
        #list of strings for different outcomes
        self.outcomes=[]
        self.logs=[]
        self.type="BET"

#these two functions are to be called outside blockchain only_______________    
    def display(self,*args):
        print(self.description)
        for i,o in enumerate(self.outcomes):
            print(f'Select {i+1} for {o}')
        print(self.closingPeriod)
        print('Current betters---------')
        total={}
        for b in self.betters:
            print(f"User {b['PublicKey'][256:264]} bet {b['Amount']} on {b['Outcome']}.")
            if b['Outcome'] in total.keys():
                total[b['Outcome']]+=b['Amount']
            else:
      

Of course, each time John or Emma bets, the amount of pebbles he has should be decreased by the amount he used on the bet.
Since we're in a blockchain, these transactions only exist if they're in a block.

If we try to check the amount each of them have, we'll see that:

In [56]:
print('John has ', JohnAccount.pebbleAmount, ' pebbles')
print('Emma has ', EmmaAccount.pebbleAmount, ' pebbles')

John has  20  pebbles
Emma has  20  pebbles


Now let's assume Alice wants to close the bet, with the winning outcome being 'Yes' of index 1.

In [57]:

cert_close = SmartContractWritingOperation(Alice.publicKey, bet1.hash(), 'closeBet', [1])
Alice.sign(cert_close)
node1.new_certificate(cert_close)

We have enough certs to forge a block, and by doing so, the previous transactions will be taken into consideration:

In [58]:
SmartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(node1.blockchain, bet1.hash())
SmartContractObject.logDump()

-----------------------------------------
[BET] Started new bet by 6c1c7cde.
[BET] Added better 66f58ca8.
[ERROR] 66f58ca8 cannot bet on a different outcome.
[BET] 66f58ca8 added more to his bet.
[BET] 66f58ca8 added more to his bet.
[BET] Added better 6c1c7cde.
[BET] 6c1c7cde added more to his bet.
[BET] 66f58ca8 added more to his bet.
[BET] Redistributing winnings...
[BET] Redistributed winnings.


And now, if we check both betters' accounts:

In [59]:
print('John has ', JohnAccount.pebbleAmount, ' pebbles')
print('Emma has ',EmmaAccount.pebbleAmount, ' pebbles')

John has  24.0  pebbles
Emma has  16  pebbles


Details concerning the blockchain:

In [10]:
node1.print_certbox()
node1.blockchain.display()

--CERTBOX 0 certs--
----
{'Genesis Key': '00000000', 'Genesis Index': 0}
{'Public Key': '30820122', 'Signature': '6122655c', 'timestamp': 1709072741383, 'type': <class 'block.Block'>}
{'Index': 1, 'Parent hash': 'b7214b8d'}
--Block certs--
{'Public Key': '30820122', 'Signature': '37602b15', 'timestamp': 1709072741308, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '5510b782', 'timestamp': 1709072741266, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '0ac44df4', 'timestamp': 1709072741348, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '7b9d947d', 'timestamp': 1709072740215, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': 'a338f5d9', 'timestamp': 1709072739172, 'type': <class 'smart_contract.SmartContractDefinition'>}
----
{'Public Key': '30820122', 'Signature': '

In [11]:
bet1.find_trending_bets(node1.blockchain)

-----------------------------------------


[{'contract_Owner': 'a0134bb8',
  'contract_desc': 'Lorum Lora',
  'total_mises': 17}]

Let's create a second bet:

In [12]:
bet2=SmartContractDefinition(Bob.publicKey,read_text_file(betFilePath))
Bob.sign(bet2)
node1.new_certificate(bet2)
sleep(1)

bet2_start = SmartContractWritingOperation(Bob.publicKey, bet2.hash(), 'startBet', [timestamp.now()+timestamp.now(), 'Yankees vs Nationals', ['Win', 'Loss', 'Draw']])
Bob.sign(bet2_start)
node1.new_certificate(bet2_start)
sleep(1)

cert11 = SmartContractWritingOperation(John.publicKey, bet2.hash(), 'addBetter', [JohnAccount, 10, 1])
John.sign(cert11)
node1.new_certificate(cert11)

cert22 = SmartContractWritingOperation(Emma.publicKey, bet2.hash(), 'addBetter', [JohnAccount, 5, 1])
Emma.sign(cert22)
node1.new_certificate(cert22)

cert2_close = SmartContractWritingOperation(Bob.publicKey, bet2.hash(), 'closeBet', [1])
Bob.sign(cert2_close)
node1.new_certificate(cert2_close)


In [13]:
bet2.find_trending_bets(node1.blockchain)

-----------------------------------------
-----------------------------------------


[{'contract_Owner': 'a0134bb8',
  'contract_desc': 'Lorum Lora',
  'total_mises': 17},
 {'contract_Owner': 'a34daa55',
  'contract_desc': 'Yankees vs Nationals',
  'total_mises': 15}]